<a href="https://colab.research.google.com/github/mc-friday/hanghaeAI/blob/main/%5B5%EC%A3%BC%EC%B0%A8%5D%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install langchain langchain-community langchain-chroma langchain-openai bs4
!pip install PyGithub

# [5주차] 심화과제: 다양한 형태의 입력을 가지는 LLM 서비스 개발

## 코드 리뷰 LLM 서비스

In [48]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from github import Github
from langchain.schema import Document

## 1. 환경 변수 설정 및 LLM 초기화

In [49]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4", api_key=openai_api_key, temperature=0)

## 2. 짧은 코드 리뷰 함수

In [50]:
def review_short_code(code_snippet):
    """
    LLM을 사용하여 짧은 코드에 대한 리뷰를 생성.
    """
    prompt = f"""
    다음은 사용자가 작성한 코드입니다:

    ```python
    {code_snippet}
    ```

    이 코드에서 문제점을 찾아 설명해주세요. 또한, 코드 품질을 개선할 수 있는 방법에 대해 간략히 제안해주세요.
    """
    response = llm.invoke([{"role": "user", "content": prompt}])
    return response.content

## 3. GitHub repository 기반 리뷰 함수

In [51]:
def review_github_repository(repo_url, access_token=None):
    """
    GitHub repository를 로드하고, RAG를 통해 문제 영역을 추출한 뒤 리뷰를 생성.
    """
    # GitHub에서 코드 로드
    loader = GitHubLoader(repo_url=repo_url, access_token=access_token)
    documents = loader.load()

    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    # 임베딩 및 벡터스토어 생성
    embedding = OpenAIEmbeddings(api_key=openai_api_key)
    vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
    retriever = vectorstore.as_retriever()

    # 문제 영역 추출
    query = """
    주어진 코드에서 잠재적인 문제 영역을 찾아주세요. 성능, 보안, 가독성과 관련된 문제들을 포함하세요.
    """
    retrieved_docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # 코드 리뷰 생성
    prompt = f"""
    다음은 프로젝트 코드의 잠재적 문제 영역입니다:

    {context}

    이 코드에서 문제를 찾아 설명해주세요. 또한, 각 문제를 해결할 수 있는 방법에 대해 간략히 제안해주세요.
    """
    response = llm.invoke([{"role": "user", "content": prompt}])
    return response.content

## 4. 메인 함수

In [52]:
def main():
    print("코드 리뷰 서비스를 시작합니다.")
    input_type = input("리뷰 대상 선택 (1: 짧은 코드, 2: GitHub repository): ").strip()

    if input_type == "1":
        code_snippet = input("리뷰할 코드를 입력하세요:\n")
        result = review_short_code(code_snippet)
        print("\n=== 코드 리뷰 결과 ===\n")
        print(result)

    elif input_type == "2":
        repo_url = input("GitHub repository URL을 입력하세요: ").strip()
        use_access_token = input("비공개 repository입니까? (y/n): ").strip().lower()

        # Access token 입력 처리
        access_token = None
        if use_access_token == 'y':
            access_token = input("Access token을 입력하세요: ").strip()

        result = review_github_repository(repo_url, access_token)
        print("\n=== 코드 리뷰 결과 ===\n")
        print(result)

    else:
        print("잘못된 입력입니다. 프로그램을 종료합니다.")

## 5. GitHubLoader 클래스

In [53]:
class GitHubLoader:
    def __init__(self, repo_url, access_token=None):
        """
        GitHubLoader 초기화
        :param repo_url: GitHub repository URL
        :param access_token: 개인 access token (optional)
        """
        self.repo_url = repo_url
        self.access_token = access_token
        self.github = Github(access_token) if access_token else Github()

    def load(self):
        """
        GitHub repository의 모든 파일 로드
        :return: Document 객체 리스트
        """
        repo_name = self.repo_url.split("github.com/")[-1]  # Repository 이름 추출
        repo = self.github.get_repo(repo_name)  # Repository 가져오기
        documents = []

        # Repository의 모든 파일 로드
        for content in repo.get_contents(""):
            if content.type == "file":  # 파일인지 확인
                file_content = repo.get_contents(content.path).decoded_content.decode("utf-8")
                documents.append(Document(page_content=file_content, metadata={"path": content.path}))

        return documents


## 6. 프로그램 실행 (Github)

In [55]:
if __name__ == "__main__":
    main()

코드 리뷰 서비스를 시작합니다.
리뷰 대상 선택 (1: 짧은 코드, 2: GitHub repository): 2
GitHub repository URL을 입력하세요: https://github.com/toss/toss-cert-examples
비공개 repository입니까? (y/n): ㅜ


<ipython-input-51-d7fba99b7337>:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(api_key=openai_api_key)



=== 코드 리뷰 결과 ===

이 코드는 프로젝트의 .gitattributes 파일로 보입니다. 이 파일은 Git이 프로젝트의 파일을 어떻게 처리해야 하는지를 정의합니다. 이 코드에서는 특정 파일 유형에 대한 diff 및 merge 동작을 설정하고 있습니다.

1. 문제: 주석 처리된 코드
   이 코드에서는 여러 파일 유형에 대한 diff 및 merge 동작이 주석 처리되어 있습니다. 이는 해당 파일 유형에 대한 기본 동작이 적용되지 않음을 의미합니다. 이로 인해 예상치 못한 동작이 발생할 수 있습니다.

   해결 방법: 필요한 동작을 정확히 알고 있다면, 주석을 제거하고 해당 동작을 활성화해야 합니다. 예를 들어, .cs 파일에 대한 diff 동작이 필요하다면 "#*.cs     diff=csharp"의 주석을 제거해야 합니다.

2. 문제: 누락된 파일 유형
   이 코드에서는 일부 파일 유형에 대한 동작만 정의되어 있습니다. 따라서 다른 파일 유형에 대한 동작이 정의되지 않았다면, Git의 기본 동작이 적용됩니다. 이로 인해 예상치 못한 동작이 발생할 수 있습니다.

   해결 방법: 프로젝트에서 사용하는 모든 파일 유형에 대한 동작을 .gitattributes 파일에 정의해야 합니다. 이렇게 하면 Git이 각 파일 유형을 어떻게 처리해야 하는지 알 수 있습니다.

3. 문제: 불필요한 설정
   이 코드에서는 이미지 파일을 이진 파일로 처리하도록 설정되어 있지만, Git은 기본적으로 이미지 파일을 이진 파일로 취급합니다. 따라서 이 설정은 불필요합니다.

   해결 방법: 불필요한 설정은 제거하는 것이 좋습니다. 이 경우, 이미지 파일을 이진 파일로 취급하는 설정을 제거해도 됩니다.

4. 문제: 문서 파일에 대한 diff 설정
   이 코드에서는 일부 문서 파일에 대한 diff 동작을 설정하고 있지만, 이 설정은 명령줄에서만 사용할 수 있습니다. 따라서 GUI 도구를 사용하는 경우 이 설정이 적용되지 않을 수 있습니다.

   해결 방법: GU

## 7. 프로그램 실행 (짧은코드)

In [56]:
if __name__ == "__main__":
    main()

코드 리뷰 서비스를 시작합니다.
리뷰 대상 선택 (1: 짧은 코드, 2: GitHub repository): 1
리뷰할 코드를 입력하세요:
def add(a, b):     return a + b

=== 코드 리뷰 결과 ===

이 코드는 기능적으로는 문제가 없습니다. 두 개의 인자를 받아 더하는 간단한 함수입니다. 그러나 코드 품질을 높이기 위한 몇 가지 제안은 다음과 같습니다:

1. **주석 추가**: 코드에 주석을 추가하여 함수의 기능을 명확하게 설명하면 다른 개발자가 코드를 이해하는데 도움이 됩니다.

2. **타입 힌트 추가**: 파이썬 3.5 이상에서는 함수의 매개변수와 반환 값에 대한 타입 힌트를 제공할 수 있습니다. 이는 코드의 가독성을 높이고, 잠재적인 오류를 미리 방지하는데 도움이 됩니다.

3. **적절한 줄바꿈과 들여쓰기**: 코드의 가독성을 높이기 위해 적절한 줄바꿈과 들여쓰기를 사용하는 것이 좋습니다.

위의 제안을 반영하여 코드를 개선하면 다음과 같습니다:

```python
def add(a: int, b: int) -> int:
    """
    두 정수를 입력받아 더한 값을 반환하는 함수
    """
    return a + b
```
